In [23]:
import shap
import numpy as np
from sklearn.linear_model import LinearRegression
from lime import lime_tabular
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
import gymnasium as gym

shap.initjs()

In [5]:
env = gym.make('MountainCarContinuous-v0')
model = PPO.load("ppo_mountain.zip",env,verbose=100)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [37]:
model.learn(total_timesteps=2000000)

# Save the trained model
model.save("ppo_mountain.zip")

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 999       |
|    ep_rew_mean     | -0.000623 |
| time/              |           |
|    fps             | 2447      |
|    iterations      | 1         |
|    time_elapsed    | 0         |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 999         |
|    ep_rew_mean          | -0.000593   |
| time/                   |             |
|    fps                  | 1556        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010238464 |
|    clip_fraction        | 0.139       |
|    clip_range           | 0.2         |
|    entropy_loss         | 5.01        |
|    explained_variance   | -2.33       |
|    learning_rate  

KeyboardInterrupt: 

In [9]:
observation, info = env.reset()

XTrain = []
YTrain = []
done = False

x = 0
for i in range(10):
    while not done and x < 900:
        action = model.predict(observation)
        XTrain.append(observation)
        YTrain.append(action[0])
    
        observation, reward, done, _, _ = env.step(action[0])
        x+=1

In [27]:
print(YTrain)

[array([-0.00150237], dtype=float32), array([-0.00225733], dtype=float32), array([-0.0017037], dtype=float32), array([-0.0014866], dtype=float32), array([-0.00184602], dtype=float32), array([-0.00336255], dtype=float32), array([-2.7602771e-05], dtype=float32), array([-0.00212547], dtype=float32), array([-0.00446618], dtype=float32), array([0.00170167], dtype=float32), array([-0.00063611], dtype=float32), array([-0.00624348], dtype=float32), array([-0.0031511], dtype=float32), array([-9.6314005e-05], dtype=float32), array([-0.0017215], dtype=float32), array([-0.00149653], dtype=float32), array([-0.00269581], dtype=float32), array([-0.00525052], dtype=float32), array([-0.00082595], dtype=float32), array([-0.00301568], dtype=float32), array([-0.00024459], dtype=float32), array([-0.00215484], dtype=float32), array([-0.00262878], dtype=float32), array([-0.00368143], dtype=float32), array([-0.00305592], dtype=float32), array([-0.00224981], dtype=float32), array([-0.00231859], dtype=float32),

In [11]:
XTest = []

observation, _ = env.reset()

x=0
while not done and x < 900:
    action = model.predict(observation)
    XTest.append(observation)

    observation, reward, done, _, _ = env.step(action[0])
    x+=1

In [19]:
def f(X):
    return model.predict(X)[0]

In [20]:
XTrain_np = np.array(XTrain)
XTest_np = np.array(XTest)

print(XTrain_np)

explainer = shap.KernelExplainer(f, XTrain_np)

shap_values = explainer.shap_values(XTest_np)
print("shap_values =", shap_values)
print("base value =", explainer.expected_value)

Using 900 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


[[-5.6808591e-01  0.0000000e+00]
 [-5.6775546e-01  3.3041020e-04]
 [-5.6709820e-01  6.5723161e-04]
 ...
 [-4.9413675e-01  2.8593675e-03]
 [-4.9149519e-01  2.6415668e-03]
 [-4.8909953e-01  2.3956678e-03]]


  0%|          | 0/900 [00:00<?, ?it/s]

shap_values = [array([[ 0.00264773,  0.00207043],
       [ 0.00021447, -0.00038384],
       [ 0.00177412,  0.00133477],
       ...,
       [-0.00051288, -0.00024828],
       [-0.00021971,  0.00014686],
       [-0.002273  , -0.00179548]])]
base value = [-0.0018566]


In [29]:
print(explainer.expected_value[0])
print(shap_values[0])
print(XTest_np[0,:])

shap_values = explainer.shap_values(XTest_np)

-0.0018565987917827443
[[ 0.00264773  0.00207043]
 [ 0.00021447 -0.00038384]
 [ 0.00177412  0.00133477]
 ...
 [-0.00051288 -0.00024828]
 [-0.00021971  0.00014686]
 [-0.002273   -0.00179548]]
[-0.59877485  0.        ]


  0%|          | 0/900 [00:00<?, ?it/s]

In [32]:
shap.force_plot(explainer.expected_value, shap_values[0], XTest_np, feature_names=["x_pos","vel"])

In [ ]:
env_test = gym.make('MountainCarContinuous-v0',render_mode="human")

obs, inf = env_test.reset()

while True:
    env_test.render()
    obs, rew, done, ter, _ = env_test.step(model.predict(observation)[0])

    if done or ter:
        obs, inf = env_test.reset()
        print("reset")